In [62]:
library(SummarizedExperiment)
library(VariantAnnotation)
library(Rtsne)
library(ggplot2)
library(gplots)
library(dplyr)
library(irlba)
library(umap)
library(igraph)
library(FNN)
vcf <- readRDS('~/Dropbox/mixed/original/Map_K10_VBC_CLI.whitelist.noindel.rds')
meta <- read.csv('~/Dropbox/mixed/meta/MetaDATAJohnes2.csv', header = TRUE)
rownames(meta) <- meta$SeqID
meta <- meta[colnames(vcf), ]
meta <- meta %>% mutate(Farms = sprintf('farm_%d', Farm))

In [4]:
GT <- geno(vcf)$GT

In [5]:
table(c(GT))
GT[GT %in% c('.')] <- '0/0' # missing values
GT[GT%in% c('0/2', '0/3')] <- '0/1'
table(c(GT))


      .     0/0     0/1     0/2     0/3     1/1 
    346 1434753    5506     199       9    5562 


    0/0     0/1     1/1 
1435099    5714    5562 

In [70]:
# We define the orphan snps as the ones that are present in only one sample as
# heterozygous or homozygous SNP
GT_freq <- do.call('rbind', lapply(
    1:nrow(GT), 
    function(i) GT[i, ] %>% factor(c('0/0', '0/1', '1/1'), labels = c('AA', 'AB', 'BB')) %>% table()
)) %>% 
    as.data.frame() %>%
    mutate(snp_id = rownames(GT)) %>%
    filter(BB > 1 & AA > 1) %>% 
    arrange(BB)

In [102]:
states <- c('MN','NY', 'PA', 'VT')
GT_AB_states <- do.call('rbind', lapply(1:nrow(GT_freq), function(i){
    j <- GT[GT_freq[i, 'snp_id'], ] %in% c('0/1') 
    (meta$State[j] %>% factor(states) %>% table()) 
})) 
colnames(GT_AB_states) <- sprintf('%s_AB', colnames(GT_AB_states))
GT_BB_states <- do.call('rbind', lapply(1:nrow(GT_freq), function(i){
    j <- GT[GT_freq[i, 'snp_id'], ] %in% c('1/1') 
    (meta$State[j] %>% factor(states) %>% table()) 
})) 
colnames(GT_BB_states) <- sprintf('%s_BB', colnames(GT_BB_states))

In [103]:
df <- cbind(GT_freq, GT_AB_states, GT_BB_states)

In [105]:
df %>% 
    arrange(AB)

AA,AB,BB,snp_id,MN_AB,NY_AB,PA_AB,VT_AB,MN_BB,NY_BB,PA_BB,VT_BB
523,0,2,Map_K10_VBC_CLI:6008_C/T,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:9890_C/T,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:10583_G/A,0,0,0,0,0,0,2,0
523,0,2,Map_K10_VBC_CLI:28335_G/A,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:49215_A/G,0,0,0,0,0,2,0,0
523,0,2,Map_K10_VBC_CLI:71631_G/A,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:77046_G/T,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:120560_C/T,0,0,0,0,2,0,0,0
523,0,2,Map_K10_VBC_CLI:122449_C/T,0,0,0,0,0,0,2,0
523,0,2,Map_K10_VBC_CLI:157436_G/T,0,0,0,0,2,0,0,0


In [95]:
table(GT['Map_K10_VBC_CLI:77507_C/A', ], GT['Map_K10_VBC_CLI:165969_C/T', ])

     
      0/0 0/1 1/1
  0/0 465   0   0
  0/1   0   6   0
  1/1   0   0  54

In [173]:
d <- data.frame(
    x = y_tsne[, 1], 
    y = y_tsne[, 2],
    n_heterozygous = (X_no_orphan > 0.1 & X_no_orphan < 0.9) %>% colSums(),
    cow_id = meta$CowID,
    type_1 = meta$Type %>% factor(),
    type_2 = meta$Type2 %>% factor(),
    farm = meta$Farm %>% factor(),
    contamination = meta$Contamination %>% factor(),
    date = meta$Date %>% factor(),
    state = meta$State %>% factor(),
    mix = meta$Mix %>% factor()
)
head(d)

,x,y,n_heterozygous,cow_id,type_1,type_2,farm,contamination,date,state,mix
MAP129,3.668753,-3.303833,29,1283,tissue,Ileum_20cm_from_IC_Valve,9,0,2006,NY,1
MAP022,2.441327,-1.987906,8,1085,fecal,fecal,9,0,2004,NY,0
MAP153,2.240757,3.020176,6,1181,tissue,LymphNode1,9,0,2006,NY,1
MAP160,-3.466697,-3.564802,2,1355,tissue,Ileum_20cm_from_IC_Valve,9,0,2006,NY,0
MAP168,3.756924,8.876470,3,1645,fecal,fecal,9,0,2006,NY,0
MAP140,-4.535493,-4.579195,12,2158,tissue,LymphNode2,9,0,2006,NY,1
